# Using RAG Fine Tuning

First, install dependencies

In [ ]:
!pip install faiss-cpu sentence-transformers jina

Imports libraries

In [1]:
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModel
from transformers import AutoModel
import faiss
import numpy as np

Load a pre-trained embedding model

In [2]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
model_name = "jinaai/jina-embeddings-v2-base-en" #8192 sequence length
model = AutoModel.from_pretrained(model_name, trust_remote_code=True) 

Embeddings

In [ ]:
documents = [ "Retrieval-Augmented Generation enhances LLMs with external knowledge.",
              "FAISS is an efficient similarity search library for vector retrieval.",
              "Sentence Transformers can convert text into numerical vector embeddings.",
              "Vector databases allow fast and scalable similarity searches.",
              "Deep learning models benefit from additional retrieved context.",
              "Large language models are trained on large datasets.",
              "Large language models can benifit from additional context from external sources.",
                ]

In [4]:
embeddings = model.encode(documents,max_length=2048) 

In [5]:
dimension = embeddings.shape[1]
print(f"Embedding dimension: {dimension}")

Embedding dimension: 768


Store embeddings in FAISS

In [6]:
index = faiss.IndexFlatL2(dimension) 
index.add(embeddings)
print(f"FAISS index size: {index.ntotal} vectors")

FAISS index size: 7 vectors


In [7]:
def retrieve_documents(query, top_k=2):
    # Convert the query into an embedding
    query_embedding = model.encode([query])

    # Search the FAISS index
    _, indices = index.search(np.array(query_embedding), top_k)

    # Retrieve top-k matching documents
    retrieved_docs = [documents[i] for i in indices[0]]
    
    return retrieved_docs

Example query

In [8]:
query = "How does FAISS help with retrieval?"
retrieved = retrieve_documents(query, top_k=2)

In [10]:
print("Retrieved Documents:")
for doc in retrieved:
    print("-", doc)

Retrieved Documents:
- FAISS is an efficient similarity search library for vector retrieval.
- Retrieval-Augmented Generation enhances LLMs with external knowledge.
